In [8]:
import pickle, os, json
import scipy.io as sio
import numpy as np
import pandas as pd
from bci_funcs import slope

# for each pID
path = '/Users/lukasgehrke/data/fastReach/data/eeglab2python'

pIDs = [12, 14, 15, 16, 17, 19, 20, 21, 22, 23]

for pID in pIDs:
    
    pID = 'sub-' + "%03d" % (pID)

    # load classifier
    filename = 'model_' + pID + '_eeg.sav'
    loaded_model = pickle.load(open(path+os.sep+pID+os.sep+'model_'+pID+'_eeg.sav', 'rb'))
    with open(path+os.sep+pID+os.sep+'bci_params.json') as f:
        bci_params = json.load(f)

    # compute features from erps
    erp = sio.loadmat(path+os.sep+pID+os.sep+'eeg_move.mat')['eeg_move']
    erp = erp[bci_params['chans'],:,:]

    feat = np.zeros((erp.shape[2], erp.shape[0]))
    for trial_ix in range(erp.shape[2]):
        feat[trial_ix, :] = slope(erp[:,249:499,trial_ix], 'linear').flatten()

    # apply classifier and get probas
    proba = loaded_model.predict_proba(feat)

    # load design
    design = pd.read_csv(path+os.sep+pID+os.sep+'design.csv')

    # save and add to design
    design['proba'] = proba[:,1]
    design.to_csv(path+os.sep+pID+os.sep+'design.csv', index=False)

KeyboardInterrupt: 